In [29]:
import tensorflow as tf
import os
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Embedding, BatchNormalization, Concatenate
from tensorflow.keras.optimizers import Adam
import numpy as np
import dill as dpickle
print(tf.__version__)
print(os.getcwd())
!ls

1.13.1
C:\Users\siddu\Documents\COURSES\ASE\Project
Clustering.ipynb
body_pp.dpkl
issues_labeller
issues_labeller.tar
predictor.ipynb
preprocess_data.ipynb
test_body_vecs.npy
test_labels.npy
test_title_vecs.npy
testdf.pkl
title_pp.dpkl
train_body_vecs.npy
train_labels.npy
train_model.ipynb
train_title_vecs.npy
traindf.pkl


In [28]:
with open('title_pp.dpkl', 'rb') as f:
    title_pp = dpickle.load(f)

with open('body_pp.dpkl', 'rb') as f:
    body_pp = dpickle.load(f)
    
#load the training data and labels
train_body_vecs = np.load('train_body_vecs.npy')
train_title_vecs = np.load('train_title_vecs.npy')
train_labels = np.load('train_labels.npy')

#load the test data and labels
test_body_vecs = np.load('test_body_vecs.npy')
test_title_vecs = np.load('test_title_vecs.npy')
test_labels = np.load('test_labels.npy')


issue_body_doc_length = train_body_vecs.shape[:]
issue_title_doc_length = train_title_vecs.shape[:]

body_vocab_size = body_pp.n_tokens
title_vocab_size = title_pp.n_tokens

num_classes = len(set(train_labels[:, 0]))

[[   0    0    0    0    0    1  171    1    3   67]
 [   0    0    0    0    0    0    0   11  450 3105]
 [   0    0    0    0    0    0   49   81    1    1]
 [   0    0    0  102  181   17   14    1  315 1047]
 [   0    0    0    0    0    0   49  434 3206  150]
 [   0    0    0    0    0    0  263 1023  112 1321]
 [   0    0    0    0   37 1297  209    5  204  733]
 [   0    0    0    0    0    0  769    8 1456  230]
 [ 240  176  149 1021   56    3    8  194  123  124]
 [   0    0    0    0    0    1  702  352  128    1]]
(1871144, 110)
(1871144, 10)


In [ ]:
body_input = Input(shape=(issue_body_doc_length,), name='Body-Input')
title_input = Input(shape=(issue_title_doc_length,), name='Title-Input')

body = Embedding(body_vocab_size, 50, name='Body-Embedding')(body_input)
title = Embedding(title_vocab_size, 50, name='Title-Embedding')(title_input)

body = BatchNormalization()(body)
body = GRU(100, name='Body-Encoder')(body)

title = BatchNormalization()(title)
title = GRU(75, name='Title-Encoder')(title)

x = Concatenate(name='Concat')([body, title])
x = BatchNormalization()(x)
out = Dense(num_classes, activation='softmax')(x)

model = Model([body_input, title_input], out)

model.compile(optimizer=Adam(lr=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

script_name_base = 'Issue_Label_v1'
csv_logger = CSVLogger('{:}.log'.format(script_name_base))
model_checkpoint = ModelCheckpoint('{:}_best_model.hdf5'.format(script_name_base),
                                   save_best_only=True)

In [ ]:
batch_size = 900
epochs = 4
history = model.fit(x=[train_body_vecs, train_title_vecs], 
                    y=train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=[(test_body_vecs, test_title_vecs), test_labels], 
                    callbacks=[csv_logger, model_checkpoint])

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    
plot_history(history)